In [1]:
import json
import pandas as pd
import numpy as np
import math
import base64
import re
import rsa

In [2]:
with open('Zad3/certs.02_2020.json') as json_file:
    data_certs = [json.loads(line) for line in json_file]

In [3]:
data_certs[0].keys()

dict_keys(['_id', 'url', 'cert_PEM', 'keyHash', 'bits', 'is_only_pub', 'key_type', 'key_PEM', 'rsa_PEM'])

In [4]:
df_certs = pd.DataFrame(data_certs, dtype='object')

In [5]:
df_certs.count()

_id            134574
url            134574
cert_PEM       134574
keyHash        134574
bits           134574
is_only_pub    134574
key_type       134574
key_PEM        134574
rsa_PEM        122030
dtype: int64

In [7]:
#remove elliptical curves (no RSA key)
df_certs = df_certs[df_certs['rsa_PEM'].notnull()]

In [11]:
#remove duplicates
df_certs = df_certs.drop_duplicates(subset='rsa_PEM', keep="first")
df_certs.count()

_id            106447
url            106447
cert_PEM       106447
keyHash        106447
bits           106447
is_only_pub    106447
key_type       106447
key_PEM        106447
rsa_PEM        106447
dtype: int64

In [15]:
df_certs.head()

,_id,url,cert_PEM,keyHash,bits,is_only_pub,key_type,key_PEM,rsa_PEM
0,{'$oid': '5e80d58b6a0f8ddedd7e244d'},https://0-9.pl,{'$binary': {'base64': 'LS0tLS1CRUdJTiBDRVJUSU...,41b027f4f3c04a8c8f907b26f1fea99e,2048,True,<class 'cryptography.hazmat.backends.openssl.r...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,-----BEGIN RSA PUBLIC KEY-----\nMIIBCgKCAQEAwJ...
1,{'$oid': '5e80d58b6a0f8ddedd7e244e'},https://01-studio.pl,{'$binary': {'base64': 'LS0tLS1CRUdJTiBDRVJUSU...,ff91bc7d9305e8675286793cf7c61705,2048,True,<class 'cryptography.hazmat.backends.openssl.r...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,-----BEGIN RSA PUBLIC KEY-----\nMIIBCgKCAQEAub...
2,{'$oid': '5e80d58b6a0f8ddedd7e244f'},https://0155oddzialsiemianowice.all.pl,{'$binary': {'base64': 'LS0tLS1CRUdJTiBDRVJUSU...,dd7a9119d13f02985eca4e3f46be7ac3,2048,True,<class 'cryptography.hazmat.backends.openssl.r...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,-----BEGIN RSA PUBLIC KEY-----\nMIIBCgKCAQEAwd...
3,{'$oid': '5e80d58c6a0f8ddedd7e2450'},https://001.apetete.pl,{'$binary': {'base64': 'LS0tLS1CRUdJTiBDRVJUSU...,b77c5505a21fa73efe817292123d5fe6,2048,True,<class 'cryptography.hazmat.backends.openssl.r...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,-----BEGIN RSA PUBLIC KEY-----\nMIIBCgKCAQEA1b...
4,{'$oid': '5e80d58c6a0f8ddedd7e2451'},https://01moto.pl,{'$binary': {'base64': 'LS0tLS1CRUdJTiBDRVJUSU...,34a6894ef9e6e85758657050b904fb84,2048,True,<class 'cryptography.hazmat.backends.openssl.r...,-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w...,-----BEGIN RSA PUBLIC KEY-----\nMIIBCgKCAQEAlz...


Example RSA public key:

##### 30
The value 30 is used to signify 'sequence';
this is a container that carries a list of DER-encoded objects.
##### 82 01 0a
how many bytes are in the DER-encoding of the object, not counting the object type and the length field
##### 02
the value 02 is used to signify 'integer'
##### 82 01 01
this is the length of the integer
##### 0a 02 82 01 01 00 c0 9f fe 42 c2 78 0e ac 01 c7 cf dd ce ef d4 73 f1 de 9c 00 5c 40 51 c0 55 e2 0b 9b 22 55 ea bf 22 1c 28 3a d4 fc 8f c8 47 92 89 4f 1c 48 ab 9c 4f 1b 56 71 d2 17 4f 62 04 b4 c3 69 2f 96 cf 5c 76 ba 21 b6 b0 0d 94 d1 50 59 3e c1 a4 b4 8a c2 71 c7 b4 c9 ad 2a 52 96 22 0b e2 65 34 d7 37 d2 4c 11 2f dd 64 91 31 c2 30 33 56 c5 c7 65 61 8e 5a cd 43 01 2e 9f 96 57 92 78 f9 70 e6 e4 bc 9e 27 59 70 f1 71 c9 14 7d e3 7e 34 16 0d 52 ee 7e 36 cf d1 de df e7 2b a1 30 64 92 da 98 bf 33 8e 2f 55 11 38 ba 86 08 50 fa e3 59 c4 1b 03 1c f7 68 3b dd a9 20 38 d7 63 b6 ed 44 42 68 bb 10 a0 5c fa 8d c9 3a 9a aa bd 82 56 24 a2 13 dd 27 5e e5 9b f2 aa bd 17 4e f8 f6 67 b1 f9 b8 71 63 63 f8 9d 90 ed be 71 5e 75 b8 b8 b8 6c 99 83 06 06 3b 2e d4 0c 09 1c 46 5d 03 99 72 c8 35 a1 ba 93
These 257 bytes are the actual integer, in bigendian format. Thi is the modulus (n)
##### 02
This signifies that the second element in the sequence encodes an integer
##### 03
This signifies that this integer is encoded in 3 bytes
##### 01 00 01
This signifies that the encoded integer is 0x10001 == 65537. This is the public exponent (e)

In [12]:
pubkey = rsa.PublicKey.load_pkcs1(df_certs['rsa_PEM'][0])

In [13]:
pubkey.n

24316650143925263798975353451544558720202243306713378986550343403861565210940154892571624037013068659592825249491774227453822920545978299912244870471108708602499675598756269531856634152865618112621215500780324266215294616712049828343544600802691338527817281043488390909388200181588113834304909211782421859896737585861389643408556662401508734297297907798442814316811652417169965395748828862866493056835925179831392143801799502870710211436897157651509053788567479183420857176900076611276902451169443429274970633204721166715865660087734594683509175824083658064998976059999980029658722935807910915895854769094938664745619

In [14]:
pubkey_list = []
for site in df_certs['rsa_PEM']:
    pubkey = rsa.PublicKey.load_pkcs1(site)
    pubkey_list.append(pubkey)

#### e is always 65537

In [18]:
numbers = []
for pubkey in pubkey_list:
    numbers.append(pubkey.n)

In [44]:
def producttree(X):
    result = [X]
    while len(X) > 1:
        X = [np.prod(X[int(i*2):int((i+1)*2)]) for i in range(int((len(X)+1)/2))]
        result.append(X)
    return result

def batchgcd_faster(X):
    counter = 1
    prods = producttree(X)
    R = prods.pop()
    while prods:
        X = prods.pop()     
        R = [R[math.floor(i/2)] % X[i]**2 for i in range(len(X))]
    return [math.gcd(r//n,n) for r,n in zip(R,X)]

In [45]:
len(numbers)

106447

In [53]:
%%time
result = batchgcd_faster(numbers[:10])

Wall time: 3.99 ms


In [54]:
%%time
result = batchgcd_faster(numbers[:100])

Wall time: 483 ms


In [66]:
%%time
result = batchgcd_faster(numbers[:5000])

Wall time: 14min 15s


In [67]:
[(i, numbers[i]) for i, e in enumerate(result) if e != 1]

[]

In [61]:
%%time
tmp = producttree(numbers[:2000])

Wall time: 2.62 s


In [65]:
len(tmp[0])

2000

## Parallel batch GCD

https://vntkumar8.github.io/docs/iciss_slide.pdf

https://books.google.pl/books?id=HTZCDwAAQBAJ&pg=PA307&lpg=PA307&dq=Parallelized+Common+Factor+Attack+on+RSA&source=bl&ots=8SJKyIomDk&sig=ACfU3U3zui_H9Amm37bWhzYn6xmh71cN3A&hl=xx-pirate&sa=X&ved=2ahUKEwiciMrN-c_oAhWM3KQKHcEMB4kQ6AEwB3oECAsQJQ#v=onepage&q&f=false